In [2]:
from pathlib import Path
import sys
parent = Path.cwd().resolve().parents[0]
sys.path.append(str(parent))
from MeanReversionBacktestClass import MRVectorBacktest as MR
import pandas as pd
import requests
import re

In [ ]:
start_date = '2022-01-01'
end_date = '2025-01-01'

# We need to ensure that, for whatever backtest window we choose, price data is available for every stock so the performance comparison is fair and measured over the same exact period. 
# Therefore, the earliest valid start date should be the latest common date for which data for all 101 assets of the Nasday 100 are available.

# Getting the yahoo finance tickers for all stocks in the Nasdaq 100

In [4]:
url = "https://en.wikipedia.org/wiki/Nasdaq-100"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/120.0 Safari/537.36"
}

html = requests.get(url, headers=headers, timeout=30).text
tables = pd.read_html(html)

components = next(df for df in tables if "Ticker" in df.columns)
tickers = components["Ticker"].tolist()

# Yahoo Finance uses '-' instead of '.' for tickers like BRK.B -> BRK-B
tickers = [t.replace(".", "-") for t in tickers]

print(len(tickers))
print(tickers[:15])

101
['ADBE', 'AMD', 'ABNB', 'GOOGL', 'GOOG', 'AMZN', 'AEP', 'AMGN', 'ADI', 'AAPL', 'AMAT', 'APP', 'ARM', 'ASML', 'AZN']


C:\Users\luciu\AppData\Local\Temp\ipykernel_31816\317895948.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


In [5]:
summary_dict = {}
for ticker in tickers: 
    bt = MR(ticker,window=20
            ,entry= 2.0, exit = 0.5,start=start_date,end=end_date)
    summary_dict[ticker] = bt.run_strategy()

print(summary_dict)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

{'ADBE': {'sharpe': 0.777542389633536, 'cagr': '17.88%', 'cstrat': '63.48%', 'crets': '-21.21%', 'alpha': '84.69%', 'params': {'entry': 2.0, 'exit': 0.5, 'window': 20}}, 'AMD': {'sharpe': 0.5965075856286609, 'cagr': '21.75%', 'cstrat': '80.04%', 'crets': '-19.60%', 'alpha': '99.65%', 'params': {'entry': 2.0, 'exit': 0.5, 'window': 20}}, 'ABNB': {'sharpe': 0.41761640211908047, 'cagr': '12.47%', 'cstrat': '42.05%', 'crets': '-23.90%', 'alpha': '65.95%', 'params': {'entry': 2.0, 'exit': 0.5, 'window': 20}}, 'GOOGL': {'sharpe': 0.5591542167080346, 'cagr': '11.31%', 'cstrat': '37.75%', 'crets': '31.03%', 'alpha': '6.72%', 'params': {'entry': 2.0, 'exit': 0.5, 'window': 20}}, 'GOOG': {'sharpe': 0.3907799823609583, 'cagr': '7.53%', 'cstrat': '24.22%', 'crets': '31.74%', 'alpha': '-7.52%', 'params': {'entry': 2.0, 'exit': 0.5, 'window': 20}}, 'AMZN': {'sharpe': 0.4892990456337993, 'cagr': '11.93%', 'cstrat': '40.03%', 'crets': '28.75%', 'alpha': '11.28%', 'params': {'entry': 2.0, 'exit': 0.5, 

In [6]:
def pct_to_float(x):
    """
    Converts pcts to decimals (float)
    """
    if isinstance(x, str) and x.strip().endswith("%"):
        return float(x.strip().replace("%", ""))
    return float(x)

def screener_top20(results: dict, metric: str, n: int = 20, ascending: bool = False) -> pd.DataFrame:
    """
    ranks by one of 'sharpe','cagr','cstrat','crets','alpha'
    ascending: False gives top (largest). True gives bottom (smallest).
    """
    rows = []
    for ticker, d in results.items():
        row = {"ticker": ticker}

        # metrics
        for k in ["sharpe", "cagr", "cstrat", "crets", "alpha"]:
            if k in d:
                row[k] = d[k]

        # params (optional)
        params = d.get("params", {})
        row["entry"] = params.get("entry")
        row["exit"] = params.get("exit")
        row["window"] = params.get("window")

        rows.append(row)

    df = pd.DataFrame(rows).set_index("ticker")

    # numeric sort key (convert percent strings to floats)
    if metric in ["cagr", "cstrat", "crets", "alpha"]:
        df[metric + "_num"] = df[metric].map(pct_to_float)
        sort_col = metric + "_num"
    else:
        df[metric + "_num"] = df[metric].astype(float)
        sort_col = metric + "_num"

    out = df.sort_values(sort_col, ascending=ascending).head(n)

    # show clean columns
    cols = ["sharpe", "cagr", "cstrat", "crets", "alpha", "entry", "exit", "window"]
    return out[cols]


# Top 20 by Sharpe

In [7]:
print("Top 20 by Sharpe")
print(screener_top20(summary_dict, "sharpe"))

Top 20 by Sharpe
          sharpe    cagr   cstrat    crets    alpha  entry  exit  window
ticker                                                                  
KLAC    1.164729  33.50%  137.12%   49.81%   87.31%    2.0   0.5      20
CSCO    1.148896  16.04%   55.96%    2.88%   53.08%    2.0   0.5      20
CDW     1.046634  15.51%   53.87%  -12.05%   65.92%    2.0   0.5      20
ADI     1.039447  21.20%   77.65%   26.64%   51.00%    2.0   0.5      20
MAR     0.985063  16.66%   58.48%   74.75%  -16.28%    2.0   0.5      20
PEP     0.960958  10.77%   35.74%   -4.18%   39.92%    2.0   0.5      20
INTU    0.920699  20.36%   73.98%    1.47%   72.51%    2.0   0.5      20
GFS     0.898735  33.27%  135.87%  -35.70%  171.57%    2.0   0.5      20
MNST    0.869635  13.34%   45.36%    8.89%   36.48%    2.0   0.5      20
ADSK    0.841646  20.49%   74.55%    4.18%   70.37%    2.0   0.5      20
PANW    0.809106  28.78%  112.95%  100.61%   12.34%    2.0   0.5      20
ADBE    0.777542  17.88%   63.48% 

# Top 20 by CAGR

In [8]:
print("Top 20 by CAGR")
print(screener_top20(summary_dict, "cagr"))

Top 20 by CAGR
          sharpe    cagr   cstrat    crets    alpha  entry  exit  window
ticker                                                                  
KLAC    1.164729  33.50%  137.12%   49.81%   87.31%    2.0   0.5      20
GFS     0.898735  33.27%  135.87%  -35.70%  171.57%    2.0   0.5      20
MRVL    0.769947  31.04%  124.27%   25.13%   99.14%    2.0   0.5      20
PANW    0.809106  28.78%  112.95%  100.61%   12.34%    2.0   0.5      20
SHOP    0.513325  25.02%   94.91%  -22.00%  116.90%    2.0   0.5      20
AMD     0.596508  21.75%   80.04%  -19.60%   99.65%    2.0   0.5      20
ADI     1.039447  21.20%   77.65%   26.64%   51.00%    2.0   0.5      20
LRCX    0.658527  20.68%   75.38%    3.25%   72.12%    2.0   0.5      20
ZS      0.529152  20.61%   75.06%  -40.23%  115.28%    2.0   0.5      20
ADSK    0.841646  20.49%   74.55%    4.18%   70.37%    2.0   0.5      20
INTU    0.920699  20.36%   73.98%    1.47%   72.51%    2.0   0.5      20
TTD     0.473421  20.23%   73.41%   

# Top 20 by Total Return

In [ ]:
print("Top 20 by Total Return (crets)")
print(screener_top20(summary_dict, "crets"))

Top 20 by Total Asset Return (crets)
          sharpe     cagr   cstrat    crets     alpha  entry  exit  window
ticker                                                                    
CEG    -0.692134  -16.74%  -41.70%  447.81%  -489.51%    2.0   0.5      20
MSTR   -0.544749  -32.23%  -68.73%  418.79%  -487.52%    2.0   0.5      20
NVDA   -0.286270   -9.19%  -25.03%  346.58%  -371.60%    2.0   0.5      20
PLTR   -0.159162   -6.63%  -18.53%  308.15%  -326.68%    2.0   0.5      20
AXON   -0.364630   -9.67%  -26.21%  288.77%  -314.98%    2.0   0.5      20
AVGO    0.581833   13.98%   47.85%  273.94%  -226.09%    2.0   0.5      20
APP    -0.915281  -39.73%  -77.97%  243.11%  -321.08%    2.0   0.5      20
BKNG    0.521079   13.06%   44.33%  103.64%   -59.32%    2.0   0.5      20
PANW    0.809106   28.78%  112.95%  100.61%    12.34%    2.0   0.5      20
PCAR    0.097361    1.46%    4.41%  100.47%   -96.06%    2.0   0.5      20
TMUS   -0.423847   -5.50%  -15.55%   96.60%  -112.15%    2.0   

# Top 20 by Strategy Total Return

In [ ]:
print("Top 20 by Strategy Total Return (cstrat)")
print(screener_top20(summary_dict, "cstrat"))

Top 20 by Strategy Total Return (cstrat)
          sharpe    cagr   cstrat    crets    alpha  entry  exit  window
ticker                                                                  
KLAC    1.164728  33.50%  137.12%   49.81%   87.31%    2.0   0.5      20
GFS     0.898735  33.27%  135.87%  -35.70%  171.57%    2.0   0.5      20
MRVL    0.769947  31.04%  124.27%   25.13%   99.14%    2.0   0.5      20
PANW    0.809106  28.78%  112.95%  100.61%   12.34%    2.0   0.5      20
SHOP    0.513325  25.02%   94.91%  -22.00%  116.90%    2.0   0.5      20
AMD     0.596508  21.75%   80.04%  -19.60%   99.65%    2.0   0.5      20
ADI     1.039445  21.20%   77.65%   26.64%   51.00%    2.0   0.5      20
LRCX    0.658527  20.68%   75.38%    3.25%   72.12%    2.0   0.5      20
ZS      0.529152  20.61%   75.06%  -40.23%  115.28%    2.0   0.5      20
ADSK    0.841646  20.49%   74.55%    4.18%   70.37%    2.0   0.5      20
INTU    0.920700  20.36%   73.98%    1.47%   72.51%    2.0   0.5      20
TTD     0.

# Top 20 by Alpha (outperformance)

In [ ]:
print("Top 20 by Alpha")
print(screener_top20(summary_dict, "alpha"))

Top 20 by Alpha
          sharpe    cagr   cstrat    crets    alpha  entry  exit  window
ticker                                                                  
GFS     0.898735  33.27%  135.87%  -35.70%  171.57%    2.0   0.5      20
WBD     0.428923  17.24%   60.84%  -58.27%  119.11%    2.0   0.5      20
SHOP    0.513325  25.02%   94.91%  -22.00%  116.90%    2.0   0.5      20
PYPL    0.608878  16.90%   59.47%  -56.22%  115.68%    2.0   0.5      20
ZS      0.529152  20.61%   75.06%  -40.23%  115.28%    2.0   0.5      20
AMD     0.596508  21.75%   80.04%  -19.60%   99.65%    2.0   0.5      20
MRVL    0.769947  31.04%  124.27%   25.13%   99.14%    2.0   0.5      20
KLAC    1.164728  33.50%  137.12%   49.81%   87.31%    2.0   0.5      20
ADBE    0.777542  17.88%   63.48%  -21.21%   84.69%    2.0   0.5      20
MU      0.594485  18.69%   66.85%  -10.43%   77.28%    2.0   0.5      20
QCOM    0.632390  17.26%   60.91%  -11.76%   72.67%    2.0   0.5      20
INTU    0.920700  20.36%   73.98%  